In [1]:
import json
import geojson
from django.contrib.gis.geos import MultiPolygon, GEOSGeometry
from tqdm.notebook import tqdm

Definir unidade a ser utilizada

In [2]:
company = Company.objects.get(name="GSO - UTPS")

DoesNotExist: Company matching query does not exist.

Obter os registros da unidade

In [3]:
records = OccurrenceRecord.objects.filter(company=company)

Obter os shapes de ZAS e ZSS

In [22]:
zas_shape = ShapeFile.objects.get(companies=company, name__icontains="ZAS")

In [23]:
zss_shape = ShapeFile.objects.get(companies=company, name__icontains="ZSS")

Verificar o número de registros que tem intersecção com cada shape (só pra ver mesmo)

In [24]:
records.filter(geometry__intersects=zas_shape.geometry).count()

4

In [25]:
records.filter(geometry__intersects=zss_shape.geometry).count()

7

In [26]:
records.count()

36

Atualizar campo de Local de acordo com as intersecções

In [28]:
records.filter(
    geometry__intersects=zas_shape.geometry
).update(place_on_dam="ZAS")

4

In [29]:
records.filter(
    geometry__intersects=zss_shape.geometry
).update(place_on_dam="ZSS")

7

Os registros que não estão nem na ZAS e nem na ZSS devem ter o Local preenchido com "NA" e serem inativados

In [11]:
status_inativo = ServiceOrderActionStatus.objects.get(companies=company, name="Inativado")

In [12]:
step_inativo = ApprovalStep.objects.get(approval_flow__company=company, name="Inativado")

In [32]:
records.exclude(
    Q(geometry__intersects=zas_shape.geometry) |
    Q(geometry__intersects=zss_shape.geometry)
).update(
    status=status_inativo,
    approval_step=step_inativo,
    place_on_dam="NA",
)

25